# <font color=blue>  Análises de Probabilidades de Evasão</font>
## <font color=black>  Menções do 1º semestre </font>

#### <font color=black> Escopo de análise delimitado entre o primeiro semestre de 2009 até o último semestre de 2018 do curso de Engenharia da Computação da Universidade de Brasília.</font>

In [57]:
#importar bibliotecas
import numpy as np
import pandas as pd
import csv
import json
import seaborn as sns
import itertools
import matplotlib.pyplot as plt

#inicialização biblioteca de 
sns.set()

In [1]:
path = 'data/ingresso_saida_2.csv'
df_in_out = pd.read_csv(path, sep =',')
df_in_out.head()
print(df_in_out.dtypes)

In [ ]:
saida = df_in_out['forma_saida'].value_counts()
total_estudantes = np.sum(saida)
saida_prob = np.divide(saida,total_estudantes)
saida_prob

In [ ]:
total_estudantes_saidos = np.sum(df_in_out.id_u.value_counts())
df_aux = df_in_out[~df_in_out.forma_saida.str.contains("Ativo")]
saida = np.sum(df_aux.id_u[df_aux.forma_saida != "Formatura"].value_counts())
prob_saida = np.divide(saida,total_estudantes)
#Refletir sobre esse calculo
#Muitos alunos evadem e até 
#seus colegas de semstre se formarem
#essa estatistica fica muito alta
print(prob_saida)

In [ ]:
path = 'agregado_semestres_21.csv'
df_disciplinas = pd.read_csv(path, sep =',')
df_disciplinas.head()

In [ ]:
def prob_aprov(disciplina):
    
        total = (df_disciplinas[(df_disciplinas.cod_disciplina == disciplina)])
        rep = total.id_tabela_ingresso[((total.mencao == "SR") | 
                                        (total.mencao == "II") | 
                                        (total.mencao == "MI") |
                                        (total.mencao == "TR") | 
                                        (total.mencao == "TJ"))]
        
        df_evadiu = (df_disciplinas[(df_disciplinas.cod_disciplina == disciplina)&(df_disciplinas.evadiu == 1)])
        evadiu = df_evadiu.id_tabela_ingresso[((total.mencao == "SR") | 
                                               (total.mencao == "II") | 
                                               (total.mencao == "MI") |
                                               (total.mencao == "TR") | 
                                               (total.mencao == "TJ"))]
        
        rep = rep.drop_duplicates()
        rep = np.sum(rep.value_counts())
        evadiu = evadiu.drop_duplicates()
        evadiu = np.sum(evadiu.value_counts())
        prob_rep_evadir = evadiu/rep
        
        return prob_rep_evadir

            
prob_aprovado_113093 = prob_aprov(113093)
prob_aprovado_118001 = prob_aprov(118001)
prob_aprovado_113034 = prob_aprov(113034)
prob_aprovado_117528 = prob_aprov(117528)
prob_aprovado_116301 = prob_aprov(116301)
prob_aprovado_118010 = prob_aprov(118010)

print(prob_aprovado_118010)
print(prob_aprovado_116301)
print(prob_aprovado_117528)
print(prob_aprovado_113034)
print(prob_aprovado_118001)
print(prob_aprovado_113093)

In [ ]:
def prob_evadiu(disciplinas,evadiu,mencao):
    
    aprovados_evadiu = df_disciplinas.id_tabela_ingresso.to_frame()
    aprovados_total = df_disciplinas.id_tabela_ingresso.to_frame()
    total_alunos = df_disciplinas.id_tabela_ingresso.to_frame()
    for dis in disciplinas:
        
        
        total = (df_disciplinas[(df_disciplinas.cod_disciplina == dis)])
        aprovados = total.id_tabela_ingresso[((total.mencao == mencao))]
        
        df_evadiu = (df_disciplinas[(df_disciplinas.cod_disciplina == dis)&(df_disciplinas.evadiu == evadiu)])
        evadiu_alunos = df_evadiu.id_tabela_ingresso[((total.mencao == mencao))]
        
        aprovados = aprovados.to_frame()
        evadiu_alunos = evadiu_alunos.to_frame()
        total = total.id_tabela_ingresso.to_frame()
        total_alunos = total_alunos.merge(total, how='inner')
        total_alunos = total_alunos.drop_duplicates()
        aprovados_total = aprovados_total.merge(aprovados, how='inner')
        aprovados_total = aprovados_total.drop_duplicates()
        aprovados_evadiu = aprovados_evadiu.merge(evadiu_alunos, how='inner')
        aprovados_evadiu = aprovados_evadiu.drop_duplicates()
        
    
    total_alunos = np.sum(total_alunos.id_tabela_ingresso.value_counts())
    #print("total " + str(total_alunos))
    aprovados_total = np.sum(aprovados_total.id_tabela_ingresso.value_counts())
    #print("reprovados "+ str(aprovados_total))
    aprovados_evadiu = np.sum(aprovados_evadiu.id_tabela_ingresso.value_counts())
    #print("r e evadiu "+ str(aprovados_evadiu))
    prob_aprovado_evadiu = aprovados_evadiu/aprovados_total
    if aprovados_total < 20:
        prob_aprovado = 0.01
    else:
        prob_aprovado = aprovados_total / total_alunos
    return prob_aprovado_evadiu


#### <font color=black> Mapas de calor da probabilidade de evasão dada reprovações nas disciplinas</font>

In [ ]:
semestre1 = [118010,118001,117528,116301,113093,113034]
mencoes = ['MI', 'II', 'SR', 'TJ', 'TR']
dis_sem1 = list(itertools.combinations_with_replacement(semestre1,2))
for men in mencoes:    
    data = []
    for dis in dis_sem1:
        aux = prob_evadiu([dis[0],dis[1]],1,men) 
        data.append([str(dis[0]),str(dis[1]),aux])
    heat = pd.DataFrame(data,columns=['dis', 'dis2', 'prob'])
    heat = heat.pivot('dis', 'dis2', 'prob')
    fig2, ax2 = plt.subplots(figsize=(6,6))
    with sns.axes_style("white"):
        ax = sns.heatmap(heat,annot=True, cmap="YlGnBu")
        ax.set_title(men)
        plt.show()
